In [28]:
#import libraries
import numpy as np
from numpy import asarray
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import gensim
import re
import joblib

In [ ]:
### section to re

In [2]:
# from os import listdir
# dfs=[]
# filepaths = ['../data/' + f for f in listdir('../data') if f.startswith('s') ]
# for f in filepaths:
#     df = pd.read_csv(f,encoding='utf-16')
#     dfs.append(df)
# df=pd.concat(dfs, ignore_index=True)

In [3]:
# len(df)

37961

In [4]:
# df.head(10)

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
5,?,1
6,' and 1 = 0 ) union all,1
7,? or 1 = 1 --,1
8,x' and userid is NULL; --,1
9,x' and email is NULL; --,1


In [5]:
#adding admin, user and password samples
# password = pd.read_table('../data/10-million-password-list-top-1000000.txt')
# password.dropna(inplace =True)


In [6]:
# to_add = password[password['123456'].str.contains('user|pass|admin|password|adminstrator')].head(500)
# to_add.rename(columns = {'123456':'Sentence'}, inplace = True)
# to_add['Label'] = 0
# to_add.head()

,Sentence,Label
0,password,0
73,pass,0
305,password1,0
409,passw0rd,0
751,passion,0


In [7]:
### cleaning
# df = df[~df['Sentence'].isna()]

In [8]:
# df2= pd.concat([df, to_add])
# df2

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
...,...,...
203242,password1981,0
203243,password@123,0
203247,passWORD,0
203248,passwolf,0


In [9]:
# df2.to_csv('../data/training.csv')

In [6]:
df = pd.read_csv('../data/training.csv',index_col=0)
xss= pd.read_csv('../data/XSS_dataset.csv', index_col =0)

In [7]:
xss.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [11]:
df2= pd.concat([df,xss])

In [12]:

def clean(text):
    pattern = re.compile(r"\d*([^\d\W]+)\d*") # removes only those numbers appearing with words but leaves individual numbers alone
    
    return pattern.sub(r"\1", text)

In [13]:
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords,token_pattern ='[a-zA-Z0-9$&+,:;=?@#|<>.^*()%!-]+')
X = vectorizer.fit_transform(df2['Sentence'].apply(lambda x:clean(x)))
y = df2['Label']
# X = vectorizer.fit_transform(df['words'])


In [14]:
# transformed_posts=pd.DataFrame(posts)

In [15]:
# posts

In [16]:
vectorizer.get_feature_names()[:10]

['!',
 '!)',
 '!<1',
 '!<1,version',
 '!<@',
 '!<@,version',
 '#',
 '#!forum',
 '#,@@version',
 '#aaa;']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
### base model logistic regression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy of logistic regression: %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of logistic regression: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of logistic regression: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of logistic regression: 0.990 
F1_score of logistic regression: 0.987 
auc_roc of logistic regression: 0.992 


In [18]:
### base model logistic regression
svc_clf = SVC(random_state=0).fit(X_train, y_train)
y_pred = svc_clf.predict(X_test)
print("Accuracy of SVC : %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of SVC: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of SVC: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of SVC : 0.991 
F1_score of SVC: 0.985 
auc_roc of SVC: 0.993 


In [19]:
dt = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = dt.predict(X_test)
print("Accuracy of Decision Tree Classifier : %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of Decision Tree Classifier: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of Decision Tree Classifier: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of Decision Tree Classifier : 0.996 
F1_score of Decision Tree Classifier: 0.994 
auc_roc of Decision Tree Classifier: 0.996 


In [50]:
from sklearn.base import BaseEstimator

    
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass
        
# Create a pipeline
pipe = Pipeline([('m', DummyEstimator())]) 

# Create param grid.

param_grid = [
    
        {
     'm': [LogisticRegression()],
     'm__penalty' : ['l1', 'l2'],
    'm__C' : np.logspace(-4, 4, 20),
    'm__solver' : ['liblinear']},
    {'m' : [DecisionTreeClassifier()],
    'm__max_depth' : list(range(2,10)),
    'm__min_samples_leaf': [5, 10, 20, 50, 100],
    'm__criterion': ["gini", "entropy"]
}
#       {'m' : [GradientBoostingClassifier()],
#     'm__max_depth' : list(range(2,10)),
#       'm__n_estimators':list(range(100,500,50))}
    
        ]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)

In [51]:
clf.fit(X,y)

Fitting 10 folds for each of 120 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 522 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 872 tasks      | elapsed:   39.7s
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:   46.8s finished


GridSearchCV(cv=10, estimator=Pipeline(steps=[('m', DummyEstimator())]),
             n_jobs=-1,
             param_grid=[{'m': [LogisticRegression(C=78.47599703514607,
                                                   solver='liblinear')],
                          'm__C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                          'm__penalty': ['l1', 'l2'],
                          'm__solver': ['liblinear']},
                         {'m': [DecisionTreeClassifier()],
                          'm__criterion': ['gini', 'entropy'],
                          'm__max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                          'm__min_samples_leaf': [5, 10, 20, 50, 100

In [52]:
clf.best_estimator_

Pipeline(steps=[('m',
                 LogisticRegression(C=78.47599703514607, solver='liblinear'))])

In [53]:
model = clf.best_estimator_

In [55]:
# save the model to disk
filename = '../model/finalized_model.sav'
joblib.dump(model, open(filename, 'wb'))

In [54]:
yhat = model.predict(X_test)
print("Accuracy of log reg: %.3f "%accuracy_score(np.rint(yhat),y_test))
print("F1_score of log reg: %.3f "%f1_score(np.rint(yhat),y_test))
print("auc_roc of log reg: %.3f "%roc_auc_score(np.rint(yhat),y_test))

Accuracy of log reg: 0.998 
F1_score of log reg: 0.997 
auc_roc of log reg: 0.998 


### END ###

In [20]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
input_dim = X.shape[1]  # Number of features
cvscores = []

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                287200    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 303,795
Trainable params: 301,747
Non-trainable params: 2,048
______________________________________________

In [21]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)

Train on 30755 samples, validate on 7689 samples
Epoch 1/10
30755/30755 [==============================] - 28s 905us/sample - loss: 0.0581 - accuracy: 0.9874 - val_loss: 0.0348 - val_accuracy: 0.9922
Epoch 2/10
30755/30755 [==============================] - 27s 891us/sample - loss: 0.0280 - accuracy: 0.9936 - val_loss: 0.0205 - val_accuracy: 0.9960
Epoch 3/10
30755/30755 [==============================] - 23s 750us/sample - loss: 0.0222 - accuracy: 0.9961 - val_loss: 0.0214 - val_accuracy: 0.9954
Epoch 4/10
30755/30755 [==============================] - 23s 748us/sample - loss: 0.0203 - accuracy: 0.9965 - val_loss: 0.0226 - val_accuracy: 0.9970
Epoch 5/10
30755/30755 [==============================] - 22s 711us/sample - loss: 0.0187 - accuracy: 0.9973 - val_loss: 0.0195 - val_accuracy: 0.9967
Epoch 6/10
30755/30755 [==============================] - 21s 697us/sample - loss: 0.0179 - accuracy: 0.9973 - val_loss: 0.0196 - val_accuracy: 0.9979
Epoch 7/10
30755/30755 [=====================

In [22]:
# Save the model
filepath = './saved_model'
save_model(model, filepath)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saved_model/assets


In [23]:
test_model = load_model(filepath, compile=True)
yhat = test_model.predict(X_test)

In [24]:
print("Accuracy of Neural network: %.3f "%accuracy_score(np.rint(yhat),y_test))
print("F1_score of Neural network: %.3f "%f1_score(np.rint(yhat),y_test))
print("auc_roc of Neural network: %.3f "%roc_auc_score(np.rint(yhat),y_test))

Accuracy of Neural network: 0.997 
F1_score of Neural network: 0.995 
auc_roc of Neural network: 0.997 


In [25]:
test = pd.read_csv('../data/Malicious_data.csv')
test_cv = vectorizer.transform(test.Payload[test['Classification']=='SQLI'])
pred= model.predict(test_cv)
np.rint(pred)

array([[1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]], dtype=float32)

In [26]:
yhat = test_model.predict(X_test)

In [27]:

print("F1_score of neural network: %.3f "%f1_score(yhat,y_test))

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
df[df['Sentence'].str.contains('Password')]

In [ ]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [ ]:
glove_vectors.most_similar('username')